<a href="https://colab.research.google.com/github/ConorD28/NBA-Research/blob/main/NBA_Predictions_Before_Semis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import math
from sklearn.decomposition import PCA
%matplotlib inline
df = pd.read_csv('NBA Finals Research_Upload.csv')
#df.drop(df.tail(37).index,
#        inplace = True)
df2 = df.drop(columns=df.columns[0], axis=1)
df_NP = df2.drop(columns=df.columns[-21:], axis=1)

In [ ]:
df2.isnull().sum().sum() #Check if there are NA values

0

In [ ]:
import scipy.stats
def correlation(dataset, threshold, target): #Function to get Pearson's correlation between input and target
  data = []
  for i in range(len(dataset.columns)):
      cor2 = dataset.iloc[:,i].corr(target) #scipy.stats.spearmanr(x, y)[0] and scipy.stats.kendalltau(x, y)[0] or Permutation Importance
      #cor2 = scipy.stats.spearmanr(dataset.iloc[:,i], target[10:30])[0]
      #cor2 = scipy.stats.kendalltau(dataset.iloc[:,i], target[10:30])[0]
      column_headers = list(dataset.columns.values)
      if(abs(cor2) > threshold):
        data.append(dataset.iloc[:,i]) #make list of columns that meet the threshold
      i = i + 1
  return data

In [ ]:
df

In [ ]:
from sklearn.model_selection import train_test_split
from numpy.random.mtrand import random_sample
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, RidgeCV, Lasso, LassoCV, ElasticNet, ElasticNetCV, MultiTaskLassoCV
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [ ]:
def Scores(y, y_pred, y_full):
  MSE = mean_squared_error(y, y_pred)
  MAE = mean_absolute_error(y, y_pred)
  Normalized_RMSE = (np.sqrt(MSE)/np.mean(y_full))*100
  Normalized_MAE = (MAE/np.mean(y_full))*100
  Avg_Normalized_Score = (Normalized_RMSE + Normalized_MAE)/2
  avg_error = (sum(abs(y_pred-y)))/10
  print(f'Avg. Normalized Score:{ Avg_Normalized_Score:.1f}%')
  #print(f'Normalized RMSE:{ Normalized_RMSE:.1f}%')
  #print(f'Normalized MAE:{ Normalized_MAE:.2f}%')
  #print(f'MAE:{ MAE:.3f}')
  #print(f'RMSE:{ np.sqrt(MSE):.3f}')
  print(f'Avg. Error:{avg_error:.1f}%')
  #print(abs(y_pred-y))
  return Avg_Normalized_Score

In [ ]:
def RLE_Model(X, y, choice, predict_df): #Function to run Ridge, Lasso, or ElasticNet model
  #X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=0) #Train/Test
  X_train, X_test, y_train, y_test = X[10:30], X[0:10], y[10:30], y[0:10]

  if(choice=="Ridge"):
    alphas = np.geomspace(1e-10, 1e10, num=100)
    m1 = RidgeCV(alphas=alphas)
    m1.fit(X_train, y_train)
    print()

  if(choice=="Lasso"):
    alphas = np.geomspace(1e-10, 1e10, num=100)
    m1 = LassoCV(alphas=alphas)
    m1.fit(X_train, y_train)
    print("Lasso alpha: ", m1.alpha_)

  if(choice=="Elastic"):
    l1_ratio = [0, 0.3, 0.5, 0.7, 0.9, 1]
    alphas = np.geomspace(1e-10, 1e10, num=100)
    m1 = ElasticNetCV(alphas=alphas, l1_ratio=l1_ratio, max_iter=100000)
    m1.fit(X_train, y_train)
    print("Elastic alpha: ", m1.alpha_)

  #print(f'Chosen alpha  {pipeline.steps[0][1].alpha_:.6f}')
  #print(f'Intercept (b) {pipeline.steps[0][1].intercept_:.6f}')
  #print(pd.Series(pipeline.steps[0][1].coef_, index=X.columns),'\n')

  #Calculate the predicted values:
  y_train_pred = m1.predict(X_train)
  #print(y_train_pred)
  print()

  y_test_pred = m1.predict(X_test)

  #Training Scores:
  Avg_N_Score_train = Scores(y_train, y_train_pred, y)
  #print()

  #Test Predictions:
  print("Test predictions:")
  #print(y_test_pred)
  #print()

  #Testing Scores:
  Avg_N_Score_test = Scores(y_test, y_test_pred, y)
  print(f'Difference of avg scores:{ Avg_N_Score_test - Avg_N_Score_train:.3f}%') #Difference between testing and traing scores to check if my bias-variance tradeoff is good
  print()

  #Predict:
  predictions = m1.predict(predict_df)

  return y_test_pred, y_train_pred, predictions

In [ ]:
from sklearn.datasets import make_regression
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split

def GBR_model(X,y, t, l, n, predict_df):
  X_train, X_test, y_train, y_test = X[10:30], X[0:10], y[10:30], y[0:10]
  reg = GradientBoostingRegressor(tol = t, learning_rate = l, n_estimators=n, random_state=0) #default: tol = 0.0001, learning rate - 0.1, 100, friedman_mse
  reg.fit(X_train, y_train)
  y_train_pred = reg.predict(X_train)
  #print(y_train_pred)

  y_test_pred = reg.predict(X_test)

  #Training Scores:
  Avg_N_Score_train = Scores(y_train, y_train_pred, y)
  #print()

  #Predictions:
  print("Test predictions:")
  #print(y_test_pred)
  #print()

  #Testing Scores:
  Avg_N_Score_test = Scores(y_test, y_test_pred, y)
  print(f'Difference of avg scores:{ Avg_N_Score_test - Avg_N_Score_train:.3f}%') #Difference between testing and traing scores to check if my bias-variance tradeoff is good
  print()

  #Predict:
  predictions = reg.predict(predict_df)
  return y_test_pred, y_train_pred, predictions

In [ ]:
from sklearn.ensemble import BaggingRegressor
import xgboost as xgb
import pickle

def BR_model(X,y):
  X_train, X_test, y_train, y_test = X[10:30], X[0:10], y[10:30], y[0:10]
  reg = BaggingRegressor(base_estimator=xgb.XGBRegressor())
  #reg = pickle.load(open("BR_model2", "rb"))
  reg.fit(X_train, y_train)
  y_train_pred = reg.predict(X_train)
  #print(y_train_pred)

  y_test_pred = reg.predict(X_test)

  #Training Scores:
  Avg_N_Score_train = Scores(y_train, y_train_pred, y)
  #print()

  #Predictions:
  print("Test predictions:")
  #print(y_test_pred)
  #print()

  #Testing Scores:
  Avg_N_Score_test = Scores(y_test, y_test_pred, y)
  print(f'Difference of avg scores:{ Avg_N_Score_test - Avg_N_Score_train:.3f}%') #Difference between testing and traing scores to check if my bias-variance tradeoff is good
  print()

  #Predict:
  #predictions = reg.predict(predict_df)
  #print(predictions)

  pickle.dump(reg, open("BR_model", "wb"))
  return y_test_pred, y_train_pred

In [ ]:
from sklearn.linear_model import SGDRegressor
#SGD Regressor:
def SGD_model(X,y, t, ep, predict_df):

  reg = SGDRegressor(max_iter=1000, tol=t, epsilon = ep) #tol = 0.001, epsilon=0.1
  X_train, X_test, y_train, y_test = X[10:30], X[0:10], y[10:30], y[0:10]
  reg.fit(X_train, y_train)
  y_train_pred = reg.predict(X_train)
  #print(y_train_pred)

  y_test_pred = reg.predict(X_test)

  #Training Scores:
  Avg_N_Score_train = Scores(y_train, y_train_pred, y)
  #print()

  #Predictions:
  print("Test predictions:")
  #print(y_test_pred)
  #print()

  #Testing Scores:
  Avg_N_Score_test = Scores(y_test, y_test_pred, y)
  print(f'Difference of avg scores:{ Avg_N_Score_test - Avg_N_Score_train:.3f}%') #Difference between testing and traing scores to check if my bias-variance tradeoff is good
  print()

  predictions = reg.predict(predict_df)
  #print(predictions)

  pickle.dump(reg, open("SGD_model", "wb"))
  return y_test_pred, y_train_pred, predictions

In [ ]:
#Keras Sequential Neural Net
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.callbacks import EarlyStopping
early_stop = EarlyStopping(monitor='loss', mode='min', verbose=1, patience=2)

def Keras_model(X,y,e, u, u2, u3, u4, u5):
  X_train, X_test, y_train, y_test = X[10:30], X[0:10], y[10:30], y[0:10]
  model = Sequential()
  model.add(Dense(u, input_dim=X_train.shape[1], activation='relu')) # Hidden 1, 60
  model.add(Dense(units=u2,activation='relu')) # Hidden 2, 30
  model.add(Dense(units=u3,activation='relu'))
  model.add(Dense(units=u4,activation='relu'))
  model.add(Dense(units=u5,activation='relu'))
  model.add(Dense(units=15,activation='relu')) #15
  model.add(Dense(units=1)) #,activation='relu'
  model.compile(loss='mean_squared_error', optimizer='adam') #adam, nadam; adamax
  model.fit(X_train, y_train, verbose=0, epochs=e, callbacks=[early_stop]); #callbacks=[early_stop]

  y_train_pred = model.predict(X_train)
  y_train_pred = y_train_pred.flatten()

  y_test_pred = model.predict(X_test)
  y_test_pred = y_test_pred.flatten()

  #Training Scores:
  Avg_N_Score_train = Scores(y_train, y_train_pred, y)
  #print()

  #Predictions:
  print("Test predictions:")
  print(y_test_pred)
  #print()

  #Testing Scores:
  Avg_N_Score_test = Scores(y_test, y_test_pred, y)
  #print(f'Difference of avg scores:{ Avg_N_Score_test - Avg_N_Score_train:.3f}%') #Difference between testing and traing scores to check if my bias-variance tradeoff is good
  print()

  #model.save('/content/drive/MyDrive/Models/Keras_Model', save_format="h5")

In [ ]:
from sklearn.tree import DecisionTreeRegressor

def DTR_model(X,y,leafs):
  X_train, X_test, y_train, y_test = X[10:30], X[0:10], y[10:30], y[0:10]
  # We introduce regularization by increasing the value of min_samples_leaf
  tree_reg_regularized = DecisionTreeRegressor(random_state=42, min_samples_leaf=leafs)
  tree_reg_regularized.fit(X_train, y_train)
  y_train_pred = tree_reg_regularized.predict(X_train) #_regularized
  #print(y_train_pred)

  y_test_pred = tree_reg_regularized.predict(X_test) #_regularized

  #Training Scores:
  Avg_N_Score_train = Scores(y_train, y_train_pred, y)
  #print()

  #Predictions:
  print("Test predictions:")
  #print(y_test_pred)
  #print()

  #Testing Scores:
  Avg_N_Score_test = Scores(y_test, y_test_pred, y)
  print(f'Difference of avg scores:{ Avg_N_Score_test - Avg_N_Score_train:.3f}%') #Difference between testing and traing scores to check if my bias-variance tradeoff is good
  print()

In [ ]:
from sklearn.svm import LinearSVR

def SVM_model(X,y,ep):
  X_train, X_test, y_train, y_test = X[10:30], X[0:10], y[10:30], y[0:10]
  svm_reg = LinearSVR(epsilon=ep, random_state=42) #default: epsilon = 0 tol=0.0001, C=1.0
  svm_reg.fit(X_train, y_train)

  #Train Predictions:
  y_train_pred = svm_reg.predict(X_train)
  #print(y_train_pred)

  #Training Scores:
  Avg_N_Score_train = Scores(y_train, y_train_pred, y)
  #print()

  #Test Predictions:
  y_test_pred = svm_reg.predict(X_test)
  print("Test predictions:")
  #print(y_test_pred)
  #print()

  #Testing Scores:
  Avg_N_Score_test = Scores(y_test, y_test_pred, y)
  print(f'Difference of avg scores:{ Avg_N_Score_test - Avg_N_Score_train:.3f}%') #Difference between testing and traing scores to check if my bias-variance tradeoff is good
  print()

In [ ]:
from sklearn.svm import SVR

def SVM_models(X,y, choice, ep, C_value, predict_df):
  X_train, X_test, y_train, y_test = X[10:30], X[0:10], y[10:30], y[0:10]
  if(choice=="rbf"):
    model = SVR(kernel="rbf", C=C_value, gamma=0.1, epsilon=ep) #0.1 default ep; 100 default C, 0.1 default gamma
    model.fit(X_train, y_train)

  if(choice=="poly"):
    model = SVR(kernel="poly", C=C_value, gamma="auto", degree=3, epsilon=ep, coef0=1) #0.1 default ep; 100 default C
    model.fit(X_train, y_train)

  if(choice == "linear"):
    model = SVR(kernel="linear", C=C_value, gamma="auto", degree=3, epsilon=ep, coef0=1) #0.1 default ep; 100 default C
    model.fit(X_train, y_train)

  #Train Predictions:
  y_train_pred = model.predict(X_train)
  #print(y_train_pred)

  #Training Scores:
  Avg_N_Score_train = Scores(y_train, y_train_pred, y)
  #print()

  #Test Predictions:
  y_test_pred = model.predict(X_test)
  print("Test predictions:")
  #print(y_test_pred)
  #print()

  #Testing Scores:
  Avg_N_Score_test = Scores(y_test, y_test_pred, y)
  print(f'Difference of avg scores:{ Avg_N_Score_test - Avg_N_Score_train:.3f}%') #Difference between testing and traing scores to check if my bias-variance tradeoff is good
  print()

  final_preds = model.predict(predict_df)

  return y_test_pred, y_train_pred, final_preds

In [ ]:
from sklearn.datasets import make_swiss_roll
from sklearn.manifold import LocallyLinearEmbedding

In [ ]:
df.iloc[:, 1106:]

In [ ]:
#Feature Importance:
scaler = StandardScaler() #MinMaxScaler, StandardScaler
X_train, X_test, y_train, y_test = df_NP[10:30], df_NP[0:10], df[10:30], df[0:10]
data_scaled = pd.DataFrame(scaler.fit_transform(X_train), columns = X_train.columns)
data_scaled.drop(["BLK_Per 100_Clutch_Plr", "BLK_Per 100_Plr"],axis=1,inplace = True) #dropped stats for EST OFF RTG

#data_scaled.drop(["PTS_Per 48",'OPP FTPerc_Per 100_Clutch','FGA_5_9 FT._Last10.2',
#                 "FGA_Above the Break 3",'PTS Above AveragePer 100 Plays',
#                 "OPP STL_Per 100_Last10", "FGM_5_9 FT._Last10.1","PACE_Per 100_vsOwnConf_Plr",
#                "PACE_Per 100_Plr"],axis=1,inplace = True) #dropped stats for EST DEF RTG
#data_correlated_df2.drop(columns=data_correlated_df2.columns[-1], axis=1,inplace=True)

data_correlated = correlation(data_scaled, 0.48, df['EST_OFFRTG_Pfs']) #
data_correlated_df = pd.DataFrame(data_correlated)
data_correlated_df2 = data_correlated_df.transpose() #Correlated inputs

#Variables Added to EST Off Rtg:
data_correlated_df2.insert(26,"PTSPer 100 Plays",data_scaled["EST. OFFRTG"])
data_correlated_df2.insert(27,"FGPerc_Corner 3",data_scaled["EST. OFFRTG"])
data_correlated_df2.insert(28,"EST. OFFRTG",data_scaled["EST. OFFRTG"])
data_correlated_df2.insert(29,"EFGPerc_A_Per 100 Plays",data_scaled["EST. OFFRTG"])
data_correlated_df2.insert(30,"PTS_Per 100_vsOwnConf_Plr",data_scaled["EST. OFFRTG"])
data_correlated_df2.insert(31,"PACE_Per 48_Clutch",data_scaled["EST. OFFRTG"])

#Variables Added to EST Def Rtg:
#data_correlated_df2.insert(21,"EST. DEFRTG",data_scaled["EST. DEFRTG"])
#data_correlated_df2.insert(22,"OPP EFGPerc_PerGM",data_scaled["OPP EFGPerc_PerGM"])
#data_correlated_df2.insert(23,"OPP TOVPerc_PerGM",data_scaled["OPP TOVPerc_PerGM"])
#data_correlated_df2.insert(24,"OPP FBPS_Per 100 Plays",data_scaled["OPP FBPS_Per 100 Plays"])
#data_correlated_df2.insert(25,"OPP PITP _Per 100 Plays",data_scaled["OPP PITP _Per 100 Plays"])

X = df_NP.loc[:, data_correlated_df2.columns] #get non scaled data with important features

#Train test split and scale:
X_train, X_test = X[10:30], X[0:10] #split non-scaled important variables
X_train_processed = pd.DataFrame(scaler.fit_transform(X_train), columns = X_train.columns)
X_test_processed = pd.DataFrame(scaler.transform(X_test), columns = X_test.columns)
correlated_scaled_data = pd.merge(X_test_processed, X_train_processed, how = 'outer')
#correlated_scaled_data_MMS = pd.merge(X_test_processed, X_train_processed, how = 'outer')

#PCA:
pca=PCA(n_components = 4)
X_train_processed_PCA = pca.fit_transform(X_train_processed)
X_train_PCA_df = pd.DataFrame(X_train_processed_PCA)
X_test_processed_PCA = pca.transform(X_test_processed)
X_test_PCA_df = pd.DataFrame(X_test_processed_PCA)
data_PCA = pd.merge(X_test_PCA_df, X_train_PCA_df, how = 'outer')
#data_PCA_MMS = pd.merge(X_test_PCA_df, X_train_PCA_df, how = 'outer')

print("Principal axes:\n", pca.components_.tolist())
print("Explained variance:\n", pca.explained_variance_.tolist())
print("Mean:", pca.mean_)

#LLE:
#X_swiss, t = make_swiss_roll(n_samples=1000, noise=0.2, random_state=42)
lle = LocallyLinearEmbedding(n_components=2, n_neighbors=15, random_state=42) #n_components=2 is default, neighbors 5 is default
X_unrolled_train = lle.fit_transform(X_train_processed)
X_train_LLE_df = pd.DataFrame(X_unrolled_train)
X_unrolled_test = lle.transform(X_test_processed)
X_test_LLE_df = pd.DataFrame(X_unrolled_test)
data_LLE = pd.merge(X_test_LLE_df, X_train_LLE_df, how = 'outer')
#data_LLE_MMS = pd.merge(X_test_LLE_df, X_train_LLE_df, how = 'outer')

X = correlated_scaled_data #try correlated_scaled_data, data_PCA, or data_LLE
y = df2['EST_OFFRTG_Pfs'] #

In [ ]:
len(data_correlated_df2.columns)

29

In [ ]:
data_correlated_df2.columns

**Estimated** **Ratings**

In [ ]:
#EST_DEFRTG_Pfs Blender: #1.8%, .2%, 1.6% avg error, .52 and added
predict_est_def_rtg = pd.read_csv('EST_DEF_RTG_Inputs.csv')

scaler = StandardScaler()
X_train_processed = scaler.fit_transform(X_train)
scaled_data = scaler.transform(predict_est_def_rtg)

pca.fit_transform(X_train_processed)
scaled_data_PCA = pca.transform(scaled_data)

scaler = MinMaxScaler()
X_train_processed_MMS = scaler.fit_transform(X_train)
scaled_data_MMS = scaler.transform(predict_est_def_rtg)

pca.fit_transform(X_train_processed_MMS)
scaled_data_PCA_MMS = pca.transform(scaled_data_MMS)

X = correlated_scaled_data
X_PCA = data_PCA
X_corr_MMS = correlated_scaled_data_MMS
X_PCA_MMS = data_PCA_MMS

preds = SVM_models(X_PCA, y, "linear", 3.3, 100, scaled_data_PCA) #1.8%, .2%, PCA,
preds2 = SGD_model(X_PCA,y, 1e-3, 0.1, scaled_data_PCA) #1.8%, .2%, PCA,
preds3 = SVM_models(X, y, "linear", 3, 150, scaled_data) #1.9%, 0%, corr
preds4 = SVM_models(X_corr_MMS, y, "linear", 3, 150, scaled_data_MMS) #1.9%, .1%, corrMMS,
preds5 = SVM_models(X_PCA_MMS, y, "linear", 3.5, 150, scaled_data_PCA_MMS) #2%, .1%, PCAMMS,
preds6 = SVM_models(X_corr_MMS, y, "poly", 3.6, 150, scaled_data_MMS) # 2.2%, 0%, corrMMS,

#Get stats on blender
train_preds = (preds[1] + preds2[1] + preds3[1] + preds4[1] + preds5[1] + preds6[1])/6

test_preds = (preds[0] + preds2[0] + preds3[0] + preds4[0] + preds5[0] + preds6[0])/6

print("Blender Train Scores then Test Scores:")
Scores(y[10:30], train_preds, y)
print()
Scores(y[0:10], test_preds, y)
print()

est_def_rtg_predictions = pd.DataFrame()
est_def_rtg_predictions["Predictions"] = (preds[2] + preds2[2] + preds3[2] + preds4[2] + preds5[2] + preds6[2])/6
est_def_rtg_predictions.to_excel("EST_DEF_RTG_Predictions.xlsx")

In [ ]:
#EST_OFFRTG_Pfs Blender: #3.2%, .9%, 3.1% avg error and .48 and added

predict_est_off_rtg = pd.read_csv('EST_OR_Inputs_4.csv')

scaler = StandardScaler()
scaler.fit_transform(X_train)
scaled_data = scaler.transform(predict_est_off_rtg)

scaler = MinMaxScaler()
scaler.fit_transform(X_train)
scaled_data_MMS = scaler.transform(predict_est_off_rtg)

X = correlated_scaled_data
X_corr_MMS = correlated_scaled_data_MMS

preds = SVM_models(X_corr_MMS, y, "poly", 3.3, 100, scaled_data_MMS) #3.1%, .9%, corr_MMS (36)
preds2 = SVM_models(X_corr_MMS, y, "linear", 3.3, 100, scaled_data_MMS) #3.2%, 1%, corr_MMS (37)
preds3 = SVM_models(X, y, "linear", 3.3, 100, scaled_data) #3.3%, 1%, corr (38)

#Get stats on blender
train_preds = (preds[1] + preds2[1] + preds3[1])/3

test_preds = (preds[0] + preds2[0] + preds3[0])/3 #3.3%, 1.1%, (39), 3.3%

print("Blender Train Scores then Test Scores:")
Scores(y[10:30], train_preds, y)
print()
Scores(y[0:10], test_preds, y)

est_off_rtg_predictions = pd.DataFrame()
est_off_rtg_predictions["Predictions"] = (preds[2] + preds2[2] + preds3[2])/3
est_off_rtg_predictions.to_excel("EST_OFF_RTG_Predictions.xlsx")